In [7]:
sequence= "MVNENTRMYIPEENHQGSNYGSPRPAHANMNANAAAGLAPEHIPTPGAALSWQAAIDAARQAKLMGSAGNATISTVSSTQRKRQQYGKPKKQGSTTATRPPRALLCLTLKNPIRRACISIVEWKPFEIIILLTIFANCVALAIYIPFPEDDSNATNSNLERVEYLFLIIFTVEAFLKVIAYGLLFHPNAYLRNGWNLLDFIIVVVGLFSAILEQATKADGANALGGKGAGFDVKALRAFRVLRPLRLVSGVPSLQVVLNSIIKAMVPLLHIALLVLFVIIIYAIIGLELFMGKMHKTCYNQEGIADVPAEDDPSPCALETGHGRQCQNGTVCKPGWDGPKHGITNFDNFAFAMLTVFQCITMEGWTDVLYWVNDAVGRDWPWIYFVTLIIIGSFFVLNLVLGVLSGEFSKEREKAKARGDFQKLREKQQLEEDLKGYLDWITQAEDIDPENEDEGMDEEKPRNMSMPTSETESVNTENVAGGDIEGENCGARLAHRISKSKFSRYWRRWNRFCRRKCRAAVKSNVFYWLVIFLVFLNTLTIASEHYNQPNWLTEVQDTANKALLALFTAEMLLKMYSLGLQAYFVSLFNRFDCFVVCGGILETILVETKIMSPLGISVLRCVRLLRIFKITRYWNSLSNLVASLLNSVRSIASLLLLLFLFIIIFSLLGMQLFGGKFNFDEMQTRRSTFDNFPQSLLTVFQILTGEDWNSVMYDGIMAYGGPSFPGMLVCIYFIILFICGNYILLNVFLAIAVDNLADAESLTSAQKEEEEEKERKKLARTASPEKKQELVEKPAVGESKEEKIELKSITADGESPPATKINMDDLQPNENEDKSPYPNPETTGEEDEEEPEMPVGPRPRPLSELHLKEKAVPMPEASAFFIFSSNNRFRLQCHRIVNDTIFTNLILFFILLSSISLAAEDPVQHTSFRNHILGNADYVFTSIFTLEIILKMTAYGAFLHKGSFCRNYFNILDLLVVSVSLISFGIQSSAINVVKILRVLRVLRPLRAINRAKGLKHVVQCVFVAIRTIGNIVIVTTLLQFMFACIGVQLFKGKLYTCSDSSKQTEAECKGNYITYKDGEVDHPIIQPRSWENSKFDFDNVLAAMMALFTVSTFEGWPELLYRSIDSHTEDKGPIYNYRVEISIFFIIYIIIIAFFMMNIFVGFVIVTFQEQGEQEYKNCELDKNQRQCVEYALKARPLRRYIPKNQHQYKVWYVVNSTYFEYLMFVLILLNTICLAMQHYGQSCLFKIAMNILNMLFTGLFTVEMILKLIAFKPKHYFCDAWNTFDALIVVGSIVDIAITEVNPAEHTQCSPSMNAEENSRISITFFRLFRVMRLVKLLSRGEGIRTLLWTFIKSFQALPYVALLIVMLFFIYAVIGMQVFGKIALNDTTEINRNNNFQTFPQAVLLLFRCATGEAWQDIMLACMPGKKCAPESEPSNSTEGETPCGSSFAVFYFISFYMLCAFLIINLFVAVIMDNFDYLTRDWSILGPHHLDEFKRIWAEYDPEAKGRIKHLDVVTLLRRIQPPLGFGKLCPHRVACKRLVSMNMPLNSDGTVMFNATLFALVRTALRIKTEGNLEQANEELRAIIKKIWKRTSMKLLDQVVPPAGDDEVTVGKFYATFLIQEYFRKFKKRKEQGLVGKPSQRNALSLQAGLRTLHDIGPEIRRAISGDLTAEEELDKAMKEAVSAASEDDIFRRAGGLFGNHVSYYQSDGRSAFPQTFTTQRPLHINKAGSSQGDTESPSHEKLVDSTFTPSSYSSTGSNANINNANNTALGRLPRPAGYPSTVSTVEGHGPPLSPAIRVQEVAWKLSSNRERHVPMCEDLELRRDSGSAGTQAHCLLLRRANPSRCHSRESQAAMAGQEETSQDETYEVKMNHDTEACSEPSLLSTEMLSYQDDENRQLTLPEEDKRDIRQSPKRGFLRSASLGRRASFHLECLKRQKDRGGDISQKTVLPLHLVHHQALAVAGLSPLLQRSHSPASFPRPFATPPATPGSRGWPPQPVPTLRLEGVESSEKLNSSFPSIHCGSWAETTPGGGGSSAARRVRPVSLMVPSQAGAPGRQFHGSASSLVEAVLISEGLGQFAQDPKFIEVTTQELADACDMTIEEMESAADNILSGGAPQSPNGALLPFVNCRDAGQDRAGGEEDAGCVRARGRPSEEELQDSRVYVSSL"
pssm_file_name = "../data/intermediate/blast/pssm_uniref50_1it/A0A0A0MR67.pssm"
PSSM_AA_ORDER = "ARNDCQEGHILKMFPSTWYV"
PSSM_AA_LIST = list(PSSM_AA_ORDER)
PSSM_AA_SET = set(PSSM_AA_ORDER)

In [20]:
with open(pssm_file_name) as pssm_file:
    next(pssm_file)
    next(pssm_file)

    amino_acids = pssm_file.readline().strip().split()[:20]
    assert (
        amino_acids == PSSM_AA_LIST
    ), f"Unexpexted amino acid order: {amino_acids}"

    # dict keeps insertion order in python 3.7+, therefore is same order as PSSM_AA_ORDER
    amino_acid_to_sum_vector = {
        amino_acid: [0.0] * 20 for amino_acid in amino_acids
    }

    sequence_pssm_file = ""

    pssm_list = list()

    for line in pssm_file:
        if line == "\n":  # end of file, before overall scores
            break

        values = line.strip().split()
        amino_acid = values[1]
        assert (
            amino_acid in PSSM_AA_SET
        ), f"unexpected amino acid in pssm file {pssm_file_name}: {amino_acid}"
        sequence_pssm_file += amino_acid

        scores = [float(score) for score in values[2:22]]
        pssm_list.append(scores)

    # Can happen for sequence conflicts, like in Q91Y77 position 5
    assert (
        sequence_pssm_file == sequence
    ), f"Sequence from PSSM file {pssm_file_name} did not match input sequence:\n{sequence_pssm_file}\n{sequence}"


In [31]:
import pandas as pd
from sklearn.preprocessing import minmax_scale
pssm = pd.DataFrame(pssm_list, columns=PSSM_AA_LIST).assign(sequence=list(sequence))

In [34]:
pssm_agg_sum = pssm.groupby("sequence").sum().loc[PSSM_AA_LIST]
pssm_agg_sum = pd.DataFrame(minmax_scale(pssm_agg_sum), columns=PSSM_AA_LIST, index=PSSM_AA_LIST)
pssm_agg_sum

,A,R,N,D,C,Q,E,G,H,I,L,K,M,F,P,S,T,W,Y,V
A,1.000000,0.244838,0.411982,0.456376,0.402367,0.406639,0.364728,0.519064,0.384722,0.184744,0.149505,0.383764,0.245583,0.076702,0.361905,0.654567,0.441128,0.020570,0.015464,0.306525
R,0.158683,1.000000,0.503226,0.475551,0.088757,0.687414,0.458910,0.348354,0.675000,0.047676,0.108911,0.805658,0.160777,0.045062,0.321905,0.414520,0.245439,0.189873,0.142612,0.100152
N,0.203593,0.387414,1.000000,0.702780,0.226331,0.556017,0.486611,0.454073,0.737500,0.059595,0.032673,0.549815,0.061837,0.071908,0.282857,0.546838,0.419569,0.090190,0.171821,0.065250
D,0.252994,0.381514,0.627650,1.000000,0.201183,0.591978,0.619575,0.449740,0.647222,0.114422,0.106931,0.531365,0.097173,0.108341,0.348571,0.471897,0.316750,0.208861,0.135739,0.095599
C,0.413174,0.363815,0.473733,0.534036,1.000000,0.481328,0.422899,0.461872,0.554167,0.379023,0.294059,0.480935,0.353357,0.269415,0.360000,0.462529,0.394693,0.444620,0.319588,0.412747
Q,0.348802,0.533923,0.545622,0.618408,0.301775,1.000000,0.580794,0.432409,0.740278,0.176400,0.192079,0.674047,0.330389,0.140940,0.398095,0.478923,0.354892,0.294304,0.264605,0.218513
E,0.238024,0.416912,0.538249,0.844679,0.000000,0.780083,1.000000,0.363085,0.669444,0.000000,0.000990,0.648216,0.053004,0.001918,0.324762,0.444965,0.233831,0.000000,0.072165,0.019727
G,0.411677,0.324484,0.544700,0.564717,0.263314,0.431535,0.397969,1.000000,0.487500,0.000000,0.015842,0.431734,0.000000,0.046021,0.307619,0.526932,0.247098,0.156646,0.000000,0.000000
H,0.378743,0.454277,0.581567,0.617450,0.400888,0.626556,0.501385,0.489601,1.000000,0.275328,0.248515,0.579336,0.298587,0.282838,0.410476,0.467213,0.398010,0.468354,0.489691,0.311077
I,0.178144,0.111111,0.211982,0.233941,0.332840,0.161826,0.173592,0.188042,0.200000,1.000000,0.504950,0.185732,0.761484,0.356663,0.128571,0.217799,0.245439,0.112342,0.178694,0.842185


In [66]:
pssm_agg_pandas_rolling = pssm.drop("sequence", axis=1).rolling(window=2, method="table").mean()
pssm_agg_pandas_rolling

ValueError: Buffer has wrong number of dimensions (expected 1, got 2)

In [71]:
from sklearn.cluster import FeatureAgglomeration
import numpy as np

agglomerator = FeatureAgglomeration(n_clusters=20, pooling_func=np.mean)

pssm_agg_sklearn_fa = pd.DataFrame(agglomerator.fit_transform(pssm.drop("sequence", axis=1).T)).T
pssm_agg_sklearn_fa = pd.DataFrame(minmax_scale(pssm_agg_sklearn_fa), columns=PSSM_AA_LIST)
pssm_agg_sklearn_fa

,A,R,N,D,C,Q,E,G,H,I,L,K,M,F,P,S,T,W,Y,V
0,0.421451,0.526445,0.374015,0.473303,0.452299,0.717678,0.526111,0.372919,0.631634,0.436351,0.472129,0.691850,0.644909,0.331622,0.380790,0.558960,0.395559,0.471917,0.450691,0.417404
1,0.000000,0.033928,0.021947,0.035897,0.181862,0.000000,0.000000,0.000000,0.430271,0.286768,0.460416,0.000000,0.352662,1.000000,0.000000,0.000000,0.000000,0.287182,0.920921,0.188523
2,0.513957,0.416516,0.439064,0.539026,0.375803,0.690577,0.564877,0.499492,0.586327,0.355185,0.385190,0.620719,0.479428,0.219186,0.451078,0.803447,0.492629,0.157646,0.306243,0.375506
3,0.333789,0.423373,0.656905,0.586698,0.306566,0.705141,0.530847,0.376878,1.000000,0.229337,0.257143,0.578170,0.341689,0.207585,0.347086,0.705745,0.408248,0.203057,0.346853,0.218348
4,0.293027,0.126299,0.088174,0.129277,0.300025,0.270116,0.175749,0.140678,0.253323,0.751103,1.000000,0.199084,1.000000,0.606943,0.157535,0.200792,0.263736,0.254523,0.376132,0.691329
5,0.376204,0.098073,0.091547,0.135450,1.000000,0.211898,0.114622,0.190615,0.171495,0.694661,0.785464,0.161622,0.837428,0.436519,0.127865,0.332795,0.299897,0.196560,0.225555,0.650702
6,0.518727,0.304082,0.282952,0.386549,0.603822,0.532114,0.395656,0.430338,0.425361,0.583329,0.620571,0.431479,0.703768,0.341259,0.387004,0.608839,0.429864,0.213450,0.334985,0.582177
7,0.395004,0.243028,0.200509,0.273336,0.423217,0.383960,0.293132,0.264100,0.338872,0.814256,0.850388,0.361247,0.877455,0.428248,0.279432,0.392091,0.377618,0.243386,0.351744,0.778692
8,0.360902,0.390187,0.447823,0.855999,0.239959,0.813053,0.890747,0.367515,0.627162,0.264996,0.280422,0.612830,0.402434,0.177841,0.341901,0.575628,0.346153,0.173852,0.297923,0.247271
9,0.450032,0.323230,0.395473,0.478490,0.333062,0.492973,0.398222,1.000000,0.433403,0.151606,0.198648,0.464017,0.269076,0.138492,0.310444,0.601609,0.273942,0.189098,0.122183,0.150055
